To do:

- Read demand data
- Mean demand --> discrete requests (random issue+pick-up time)
    - Is there any demand in this running instance for this od-pair?
    - If there is demand > 0 for an od, then put it at random points during the peak hour (=attach a pick-up time between 0 and 60)
- Import OD-matrix (travel times)

In [2]:
import pandas as pd
import numpy as np
import pickle

In [47]:
#lambdapeak: Mean demand (arrivals/min.) to travel towards the city center during peak hours
#mupeak: Mean demand (arrival/min.) to travel towards the terminal during peak hours

lambdapeak = pd.read_excel('Data/Demand rates/DemandInputSmall3.xlsx',sheet_name=0,header=0, index_col=(0,1), dtype=float)
mupeak = pd.read_excel('Data/Demand rates/DemandInputSmall3.xlsx',sheet_name=1,header=0, index_col=(0,1), dtype=float)

In [99]:
# init parameters:

size = 'small' # incorportate these in the function!
time_of_day = 'peak'
scenario = 1

In [88]:
#TODO: test on the bigger instances!

def get_scenario_mean_demand(direction, scen): #add size as a parameter later
    if direction == 'city':
        df = lambdapeak
        n = len(df.iloc[0])
        a = 1

        return df.loc[(scen,1):(scen,n),1:n]    

    else: 
        df = mupeak 
        n = len(df.iloc[0])
        a = n
    
        return df.loc[(scen,a):(scen,2*n-1),1:2*n-1]    
    

In [101]:
get_scenario_mean_demand('city', scenario)
#get_scenario_mean_demand('terminal', scenario)

1    2    3
1 1  0.0  0.1  4.0
  2  0.0  0.0  4.0
  3  0.0  0.0  0.0

In [109]:
def convert_md_todict(df_meandemand_city, df_meandemand_terminal, scen):
    n = len(df_meandemand_city.iloc[0])
    demand_dict = {}
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            demand_dict[(i,j)] = df_meandemand_city.loc[(scen,i),j]
            
    for i in range(n,2*n):
        for j in range(n,2*n):
            demand_dict[(i,j)] = df_meandemand_terminal.loc[(scen,i),j]
    
    return demand_dict
    

In [113]:
#calculate whether for this loading instance there are any requests per od-pair

small_meandemand = convert_md_todict(lambdapeak, mupeak, scenario)
#small_meandemand

{(1, 1): 0.0,
 (1, 2): 0.1,
 (1, 3): 4.0,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 4.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.15,
 (3, 5): 0.05,
 (4, 3): 0.0,
 (4, 4): 0.0,
 (4, 5): 0.0233333333333333,
 (5, 3): 0.0,
 (5, 4): 0.0,
 (5, 5): 0.0}

In [115]:
# this function generates the discrete requests 
# Q: I've made it now so that every OD-pair where there is a mean demand > 0 has at least 1 request
# I now have a lot of requests close to each other, at which point can I aggregate?

peak_hour_duration = 60

def generate_requests(mean_demand):

    requests = {}

    for od in mean_demand.keys():
        if mean_demand[od] > 0:
    
            requests_od = []
            requests[od] = requests_od
            requests[od].append(0)

            t=0

            while t < peak_hour_duration:
                delta_t = np.random.exponential(1/mean_demand[od], 1)[0]  #the time before the next passenger arrives is sampled from an exponential distr.                              
                t += delta_t
            
                if t > 60:
                    break
                else:
                    requests[od].append(t) #if no request yet is at that time

    return requests

small_requests = generate_requests(small_meandemand)
small_requests


{(1, 2): [0,
  3.6108029319004,
  4.365002508538526,
  5.536333826834273,
  10.870080936767177,
  41.601026494179635,
  47.25680720117092],
 (1, 3): [0,
  0.1405253946682855,
  0.3008053031193789,
  0.7087078624385249,
  0.7439959185117421,
  1.2445215402168945,
  1.412913053205516,
  1.5565861684661018,
  1.8374142985432782,
  1.9345055102434092,
  1.9721738220005478,
  2.0483656845634206,
  2.0707184119568462,
  2.1003603355500418,
  3.5736602480541277,
  3.7716517265724474,
  3.9384885311459894,
  4.171225834040289,
  4.185963796623194,
  4.585094360187272,
  4.621385184894214,
  4.704247445331295,
  5.073177364671289,
  5.155665556608642,
  5.5442579175012625,
  5.864476490591876,
  5.869466064798351,
  5.903505422007471,
  5.962236800885734,
  5.993848357962967,
  6.4925260803686635,
  6.7066266562850965,
  6.712661965040527,
  7.674725808653349,
  7.678964282855333,
  8.06298344187591,
  8.084350267843195,
  8.574646619978974,
  8.58811505497317,
  9.30520203974858,
  9.400185828

In [116]:
requests = open("Data/requests.pkl", "wb")
pickle.dump(small_requests, requests)
requests.close()

In [118]:
# create individual requests. R = ((o,d), pickuptime)

lst_of_ind_requests = []

for k in small_requests.keys():
    for v in small_requests[k]:
        lst_of_ind_requests.append((k, v))
        
len(lst_of_ind_requests)

525